In [1]:
import pandas as pd

df = pd.read_parquet("../clickhouse/dbtest.parquet")

In [2]:
import snowflake.connector

password = ""
account_name = ""
username = "tbenthompson"

In [7]:
# con = snowflake.connector.connect(
#     user=username,
#     password=password,
#     account=account_name
# )
# con.cursor().execute("""
#     CREATE WAREHOUSE IF NOT EXISTS imprint_warehouse WITH
#         WAREHOUSE_SIZE = 'X-SMALL'
#         WAREHOUSE_TYPE = 'STANDARD'
#         AUTO_SUSPEND = 60
#         AUTO_RESUME = TRUE
#         MIN_CLUSTER_COUNT = 1
#         MAX_CLUSTER_COUNT = 1
#         INITIALLY_SUSPENDED = TRUE
#         COMMENT = 'Imprint Warehouse';
# """)
# con.cursor().execute("""
#     CREATE DATABASE IF NOT EXISTS imprint_db;
# """)
df_chunk = df[:1000].rename(lambda x: x.upper(), axis=1)
df_chunk.shape

(1000, 69)

In [9]:
import snowflake.connector.pandas_tools

In [10]:
from sqlalchemy import create_engine

engine = create_engine(
    f"snowflake://{username}:{password}@{account_name}/imprint_db/public?warehouse=imprint_warehouse"
)

In [11]:
df_chunk.columns

Index(['ALPHA0', 'ORIG_LAM', 'B_LAM_0', 'B_LAM_1', 'B_LAM_2', 'B_LAM_3',
       'B_LAM_4', 'B_LAM_5', 'B_LAM_6', 'B_LAM_7', 'B_LAM_8', 'B_LAM_9',
       'B_LAM_10', 'B_LAM_11', 'B_LAM_12', 'B_LAM_13', 'B_LAM_14', 'B_LAM_15',
       'B_LAM_16', 'B_LAM_17', 'B_LAM_18', 'B_LAM_19', 'B_LAM_20', 'B_LAM_21',
       'B_LAM_22', 'B_LAM_23', 'B_LAM_24', 'B_LAM_25', 'B_LAM_26', 'B_LAM_27',
       'B_LAM_28', 'B_LAM_29', 'B_LAM_30', 'B_LAM_31', 'B_LAM_32', 'B_LAM_33',
       'B_LAM_34', 'B_LAM_35', 'B_LAM_36', 'B_LAM_37', 'B_LAM_38', 'B_LAM_39',
       'B_LAM_40', 'B_LAM_41', 'B_LAM_42', 'B_LAM_43', 'B_LAM_44', 'B_LAM_45',
       'B_LAM_46', 'B_LAM_47', 'B_LAM_48', 'B_LAM_49', 'TWB_MIN_LAM',
       'TWB_MEAN_LAM', 'TWB_MAX_LAM', 'SIM_SIZES', 'TODO', 'GRID_PT_IDX',
       'THETA_0', 'RADII_0', 'THETA_1', 'RADII_1', 'THETA_2', 'RADII_2',
       'THETA_3', 'RADII_3', 'NULL_TRUTH_0', 'NULL_TRUTH_1', 'NULL_TRUTH_2'],
      dtype='object')

In [28]:
%%timeit
df_chunk.to_sql(
    "TILES",
    engine,
    index=False,
    method=snowflake.connector.pandas_tools.pd_writer,
    if_exists="append",
)

<magic-timeit>:1: UserWarning: The provided table name 'TILES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.


KeyboardInterrupt: 

In [29]:
con = snowflake.connector.connect(
    user=username,
    password=password,
    account=account_name,
    database="imprint_db",
    warehouse="imprint_warehouse",
    schema="public",
)

In [30]:
%%timeit
con.cursor().execute("select count(*) from tiles").fetchall()

The slowest run took 17.37 times longer than the fastest. This could mean that an intermediate result is being cached.
362 ms ± 457 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%time
con.cursor().execute("select 1;").fetchall()

CPU times: user 2.76 ms, sys: 1.41 ms, total: 4.17 ms
Wall time: 64.6 ms


[(1,)]

In [32]:
con.commit()

In [40]:
df_chunk = df[:1000000].rename(lambda x: x.upper(), axis=1)

In [42]:
%%time
success, nchunks, nrows, output = snowflake.connector.pandas_tools.write_pandas(
    con, df_chunk, "TILES", database="IMPRINT_DB", schema="PUBLIC", compression="gzip"
)
success, nchunks, nrows

CPU times: user 3.34 s, sys: 113 ms, total: 3.45 s
Wall time: 56.5 s


(True, 1, 1000000)